# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Case Study: EDA Air Quality Data

## Learning Objectives

At the end of the case study, you will be able to

* understand various steps in exploratory data analysis
* implement those steps on Beijing air quality data which is a time series data
* know how to gain insights using these steps as a part of exploratory data analysis 

## Information

#### Dataset Description

In this assignment tutorial, we will be working on 'Beijing's Air Quality Dataset' which is a time series dataset of hourly air quality.

This data set includes hourly air pollutants data from a nationally-controlled air-quality monitoring site (Dingling Station). The air-quality data are from the Beijing Municipal Environmental Monitoring Center. The meteorological data in each air-quality site are matched with the nearest weather station from the China Meteorological Administration. The time period is from March 1st, 2013 to February 28th, 2017. Missing data are denoted as NAN.

**Attribute Information**

1. No: row number
2. year: year of data in this row
3. month: month of data in this row
4. day: day of data in this row
5. hour: hour of data in this row
6. PM2.5: PM2.5 concentration (ug/m^3)
7. PM10: PM10 concentration (ug/m^3)
8. SO2: SO2 concentration (ug/m^3)
9. NO2: NO2 concentration (ug/m^3)
10. CO: CO concentration (ug/m^3)
11. O3: O3 concentration (ug/m^3)
12. TEMP: temperature (degree Celsius)
13. PRES: pressure (hPa)
14. DEWP: dew point temperature (degree Celsius)
15. RAIN: precipitation (mm)
16. wd: wind direction
17. WSPM: wind speed (m/s)
18. station: name of the air-quality monitoring site



### Importing required packages

In [ ]:
# A tool/package to download files from the web
!pip -q install download                                 

In [ ]:
' filesystem_spec, a specification for pythonic filesystems, to produce a template or specification for a file-system interface '
!pip -q install fsspec                                                   

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
from download import download
from sklearn.impute import SimpleImputer
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

### EDA Explained With Time Series Dataset

In [ ]:
#@title Download Dataset
!wget -q https://cdn.iisc.talentsprint.com/CDS/Datasets/PRSA_Data_Dingling_20130301-20170228.csv

In [ ]:
# reading the .csv file of one monitoring station (Dingling, Changping District) of Beijing
df = pd.read_csv('PRSA_Data_Dingling_20130301-20170228.csv', encoding='ISO-8859-1')

In [ ]:
# displaying the data
df

So, from above, we can say that it is time-series data. The data is from 1st March 2013 to 28th February 2017. From the `hour column`, it is clearly seen that the data is recorded for every hour of the day. So, the data is ordered at the time. Here, we are taking the target variable as $PM_{2.5}$ as it is the key pollutant that affects the respiratory health of the ambient environment. Other variables are also there, which include other pollutants and meteorological parameters.

In [ ]:
# data information 
df.info()

In [ ]:
# defining a function for date time
def convert_to_date(x):
  return datetime.strptime(x, '%Y %m %d %H')

In [ ]:
aq_df = pd.read_csv('PRSA_Data_Dingling_20130301-20170228.csv', parse_dates=[['year', 'month', 'day', 'hour']], date_parser=convert_to_date, keep_date_col=True)

In [ ]:
# first five rows of the dataset
aq_df.head()

So, we created a single `concatenate` column which consists of year, month, date, and time, respectively. It is convenient that we make such a column in the dataset because with that, we can take any value within a particular time interval using the `pandas .iloc` function, which we will see in next sections.

In [ ]:
aq_df.info()

Now, we convert `month` to numeric for monthly basis data analysis.

In [ ]:
# converting month to numeric value data
aq_df['month']=pd.to_numeric(aq_df['month'])

Now, we are going to print the number of rows, the number of columns (or features), the number of missing values, and the number of unique values in our dataset.

In [ ]:
# diplaying the number of rows, columns, features, missing/null values and unique values in the dataset
print("Rows   :", aq_df.shape[0])
print("Columns    :", aq_df.shape[1])
print("\nFeatures  :", aq_df.columns.tolist())
print("\nMissing values  :", aq_df.isnull().any())
print("\nUnique values  :", aq_df.nunique())

Thus, as of now, we have 19 columns, 35064 rows in our dataset. True and False indicates that the feature has missing values and below that the number of unique values in each feature is shown.

In [ ]:
# descriptive statistics
aq_df.describe()

From above, we can make basic interpretations of data by seeing standard deviation, mean, minimum, maximum, etc., values. E.g., the temperature in the city can take a minimum value of -16 and a maximum value of 41, which suggests that the summers are moderately hot and the winters are too cold.

In [ ]:
# making a copy of data unindexed
aq_df_non_index= aq_df.copy()

In [ ]:
# setting index as yy mm dd hh format
aq_df = aq_df.set_index('year_month_day_hour')

In [ ]:
aq_df.index

Hence, we set our index as the date time stamp as a lot of plots for data visualization require, data indexing. Also, for subsetting and filtering data indexing is required.



In [ ]:
# first five rows 
aq_df.head()

In [ ]:
# taking values from 1st march, 2013 to 5th march, 2013
aq_df.loc['2013-03-01':'2013-03-05']

In [ ]:
# taking values from 2013 to 2015 ,i.e., two year of data
aq_df.loc['2013':'2015']

**Note:** We can do the above operations using pandas functions also without indexing. However, indexing makes this kind of operation fast and efficient. Also, some of the functions that we used for plotting considers data time to plot the data.

Now, in the next section, we will take $PM_{2.5}$ out of the data frame as we select it as our target variable.

In [ ]:
# taking PM2.5 from the data frame
pm_dt = aq_df['PM2.5']
pm_dt.head()

In [ ]:
# plotting PM2.5 data from 2013 to 2017
pm_dt.plot(grid=True)

Since there are a lot of data points, we cannot infer the graph here. Still, we can say that at the beginning and end of every year the spikes increase.

Now, we take only one year to visualize $PM_{2.5}$.

In [ ]:
# taking the year 2015
aq_df_2015 = aq_df.loc['2015']
# taking PM2.5 from above data made
pm_dt_2015 = aq_df_2015['PM2.5']
# plotting variation of PM2.5 for the year 2015
pm_dt_2015.plot(grid=True)

So, from the above plot, we can infer that the $PM_{2.5}$ levels are high in Jan, Feb, Mar, and after that, a decreasing trend can be seen, which again increases in Oct, Nov, and Dec.

However, is this pattern repeating every year? We can see that by plotting for the year 2014.

In [ ]:
# taking the year 2014
aq_df_2014 = aq_df.loc['2014']
pm_dt_2014 = aq_df_2015['PM2.5']
pm_dt_2014.plot(grid=True)

Thus, we can see that in 2014 also almost the same trend of $PM_{2.5}$ levels is appearing. 

From this, we can infer that there is some seasonality in the concentration trend of $PM_{2.5}$. In the month of Jan-Mar and Oct-Dec increase in the levels of $PM_{2.5}$ is recorded, and between them (Apr, Jun, Jul, Aug, Sept), there is a low level of $PM_{2.5}$ recorded. 

Thus, we can say that the concentration levels of $PM_{2.5}$ have some seasonality. Does it correlate with the winter season? We will see that in the following sections.

The above plots are pretty tiresome. We need to plot individually for every year and between years to visualize our data. Is there a better method where we can plot all the data and do the individual analysis? Yes, we can do that.

Python has a library for that which is `Plotly.express`.

In [ ]:
fig = px.line(aq_df_non_index, x = 'year_month_day_hour', y = 'PM2.5', title='PM2.5 with slider')

# visible the slider
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Here, using the slider we can narrow down our data and visualize it rather than creating a separate plot for each year, the month of a year, days of a month, and hourly basis. This makes the visualization even more convenient.

In [ ]:
fig = px.line(aq_df_non_index, x = 'year_month_day_hour', y = 'PM2.5', title='PM2.5 with slider')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
                      dict(count=1, label='y', step='year', stepmode='backward'),
                      dict(count=2, label='2y', step='year', stepmode='backward'),
                      dict(count=3, label='3y', step='year', stepmode='backward'),
                      dict(step='all')
        ])
    )
)
fig.show()

The above plot is the same, just only one difference here. We are providing buttons for years as 1 year (1y), 2 years (2y), and 3 years (3y). We can change these buttons to month or day by passing 'month' and 'day' instead of the year in `step` argument.

Now, we are overlapping two years (2014 and 2015) data for $PM_{2.5}$ and visualize the resulting plot.

In [ ]:
df_2014 = aq_df['2014'].reset_index()
df_2015 = aq_df['2015'].reset_index()
# removing the hour using lambda function
df_2014['month_day_hour']=df_2014.apply(lambda x : str(x['month'])+"-"+x['day'], axis=1)
df_2015['month_day_hour']=df_2015.apply(lambda x : str(x['month'])+"-"+x['day'], axis=1)
plt.plot(df_2014['month_day_hour'], df_2014['PM2.5'])
plt.plot(df_2015['month_day_hour'], df_2015['PM2.5'])
plt.legend(['2014','2015'])
plt.xlabel('Month')
plt.ylabel('PM2.5')
plt.title('Air Quality plot of PM2.5 for the year 2014 and 2015')

So, we can see that the two plots overlaid on one another. This means that the variation of $PM_{2.5}$ is almost identical in both years. We can infer from here that the variation throughout the year for different years is identical. 

**Note:** The above plot is based on Month and Day level Variations for 2014 and 2015.

Now, we will group the data by month in the next section.

In [ ]:
aq_df['2014':'2016'][['month', 'PM2.5']].groupby('month').describe()

So, from here also, we can see that the mean variation of $PM_{2.5}$ increases from Jan to Apr and then decreases and then again increases from Oct to Dec. This descriptive analysis supports the plot we made in the above sections.

Now, we will group the variation of $PM_{2.5}$ with temperature. We are taking the maximum value of $PM_{2.5}$ and the minimum and maximum value of temperature. So, we are going from a univariate analysis of $PM_{2.5}$ to a bivariate analysis.

In [ ]:
aq_df['2014':'2016'][['month', 'PM2.5', 'TEMP']].groupby('month').agg({'PM2.5':['max'], 'TEMP':['min', 'max']})

In [ ]:
# taking 2015 data
aq_df_2015=aq_df['2015']
# PM2.5 and Temperature variation in 2015
pm_dt_2015=aq_df_2015[['PM2.5', 'TEMP']]
# plotting the variations
pm_dt_2015.plot(subplots=True)

From the above variations, it is confirmed that in the winter season Jan to Mar and from Oct to Dec the variation of $PM_{2.5}$ shows an increasing trend, and the high values of $PM_{2.5}$ concentration are seen and in between from Apr to Sept and the decreasing trend can be seen and very low levels of $PM_{2.5}$ is recorded.

If we are from countries like China, Pakistan, India, Bangladesh, we can infer why this is happening. However, it will require domain knowledge.

Here in the given [article](https://weather.com/en-IN/india/science/news/2018-10-30-why-do-pollution-levels-skyrocket-during-winter), you can find why $PM_{2.5}$  has such trend or seasonality in its variations.

Now, we plot a histogram for $PM_{2.5}$ and temperature and see how it looks like.

In [ ]:
aq_df[['PM2.5', 'TEMP']].hist()

So, we can see that in the case of $PM_{2.5}$ the frequency of lower concentration is very as compared to higher levels. 

Moreover, in temperature, we can see a bi-modal distribution plot, which means it has two spikes, one in winter and one during summers.

So, to look at the temperature, we will plot a density curve to observe the two spikes.

In [ ]:
aq_df['TEMP'].plot(kind='density')

So clearly, we see two spikes, one at a lower temperature in the winter season and one at a higher temperature means in summers.

Now, we are going to plot the "lag plots." Firstly, what are **LAG PLOTS**?

The lag plot is a special type of scatter plot with two variables. The abscissa is the current time and the ordinate shows the lagged period. Now, by default, the lag period is 1, and it is called the first-order lag plot. When the lag period is 2 it is called the second-order lag plot and so on. Lag plots give much insight into our data. The linear shape of the lag plot suggests that an Autoregressive model is probably the better choice to model the data. The Lag plot can be used in multiple cases like checking the linearity in the data, checking the outliers, we can check randomness in the data, and also if there is a serial correlation or autocorrelation.

Let us plot different order lag plots now.



In [ ]:
# first order lag plot
# here our data is collected on the hourly basis
# so lag = 1 means the plot between current hour and preceding hour
pd.plotting.lag_plot(aq_df['TEMP'], lag=1)

Here, we can see that there is a linear correlation, which is also justifiable as the temperature cannot change much on an hourly basis mostly.

In [ ]:
pd.plotting.lag_plot(aq_df['TEMP'], lag=10)

So, after every 10 hour, there is a serious change in the temperature and that's why there is no correlation seen here.

In [ ]:
pd.plotting.lag_plot(aq_df['TEMP'], lag=24)

After lagging 24 hours means one day, we can observe that the data is correlated but not like that in 1-hour lag. Moreover, we can see an increasing order plot means the autocorrelation is positive here (the data is going from bottom left to top right). 

Note that here we are talking about **autocorrelation**, which means correlation within itself. Correlation typically means the relationship between two variables like spearman and Pearson's correlation which we will see further in the next sections.

One more important use case of **Lag Plot** is to observe stationarity in the data. Whether the data is stationary or not.



Now, we will plot the lag plot for one year.

Here we will take 24*365 = 8640 lag period. Meaning, 24 hours for a day and 365 for the number of days in a year.

In [ ]:
pd.plotting.lag_plot(aq_df['TEMP'], lag=8640)

We can see that the data is correlated, the data is spread out but still, it is a centered data. Thus, we can say that the temperature variation in trend is the same during the year.

In [ ]:
# half yearly variation plot of temperature
pd.plotting.lag_plot(aq_df['TEMP'], lag=4320)

Here, we can observe that the data is negatively correlated which is justifiable. As the temperature trend changes from summer to winter.

In [ ]:
# quaterly variation
pd.plotting.lag_plot(aq_df['TEMP'], lag=2160)

After every quarter of a year, the temperature changes a lot. Thus, there is no correlation seen here.

### Multivariate plot

Let's move from bivariate to multivariate plot. Here, along with temperature we will now add pressure.

In [ ]:
pm_dt_2015 = aq_df_2015[['PM2.5', 'TEMP', 'PRES']]
pm_dt_2015.plot(subplots=True)

Here, we can see that, as temperature is more pressure is less means they both are negatively correlated. 

In [ ]:
# plotting multiple weather related variables with PM2.5
multi_data = aq_df[['PM2.5', 'TEMP', 'PRES', 'DEWP', 'RAIN']]
multi_data.plot(subplots=True)

Now, we will plot the other pollutants along with $PM_{2.5}$.

In [ ]:
multi_data=aq_df[['SO2','NO2','O3','CO','PM2.5']]
multi_data.plot(subplots=True)

We can see that $PM_{2.5}$ and $CO$ having almost the same trend in variation and $PM_{2.5}$ and $O_3$ have opposite trend in variation. Means, in times when $PM_{2.5}$ levels are high that time $O_{3}$ levels are quite low and vice-versa.

In [ ]:
aq_df['2014':'2015'][['PM2.5', 'O3']].plot(figsize=(13,6.5), linewidth=3, fontsize=14)
plt.xlabel('year_month_day_hour', fontsize=20);

Here, we can clearly infer the type of relation between $PM_{2.5}$ and $O_3$ trends. In months where  $PM_{2.5}$ levels are high $O_3$ levels are low and vice-versa.

### Checking and Imputing Null Values in the data

If the data has null values in it, we can either drop them or impute them. In the case of time series analysis, we cannot drop the null value data point. This is because if we drop those data points, the continuity of our data is hindered. So, the idea is to impute the null values with a standard value of the observation. There are statistical observations like mean value, median values, mode value, etc., by which we can impute null values and depend on the type of problem we are dealing with.

In [ ]:
# observing data of the year 2015
aq_df_2015

The **NAN** values in the dataset represent the null values. This may happen because sometimes the sensor or the recording station is not working or under maintenance so there is no observation recorded for that particular day or time.

In [ ]:
# this gives true or false
# true: means null values present, false: null values absent
aq_df.isnull().values.any()

In [ ]:
# the total count of null values in each observation
aq_df.isnull().sum()

#### Imputing Null/Missing Values

In [ ]:
df_1 = aq_df_2015.drop(['wd', 'station'], axis = 1)
df_1

In [ ]:
# example of imputing missing values using Scikit-Learn

# retrieve the numpy array
values = df_1.values
# define the imputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# transform the dataset
transformed_values = imputer.fit_transform(values)

df_1

### Correlation plot between Features

In [ ]:
g = sns.pairplot(aq_df[['SO2','NO2','O3','CO', 'PM2.5']])

By this, we can observe the correlation between the features.

We can also find the values of correlation by using the pearson correlation matrix.

In [ ]:
aq_pear_corr = aq_df[['SO2','NO2','O3','CO','PM10', 'PM2.5']].corr(method='pearson')

In [ ]:
aq_pear_corr

Thus, we can interpret that $PM_{2.5}$ is highly correlated with $CO$ and $PM_{10}$ and moderately correlated with $NO_{2}$.

For better observation we can  plot a heatmap.

In [ ]:
g = sns.heatmap(aq_pear_corr, vmax=6, center=0,
                square=True, linewidths=0.5, cbar_kws={'shrink': 0.5}, annot=True, fmt='0.2f', cmap='coolwarm')
g.figure.set_size_inches(10, 10)


plt.show()

### Checking for outliers box plot

A box plot is a method for graphically depicting groups of numerical data through their quartiles. The box extends from the Q1 to Q3 quartile values of the data, with a line at the median (Q2). The whiskers extend from the edges of box to show the range of the data. The position of the whiskers is set by default to 1.5 * IQR (IQR = Q3 - Q1) from the edges of the box. Outlier points are those past the end of the whiskers.

Any data point outside this range is considered as outlier and should be removed for further analysis.

The concept of quartiles and IQR can best be visualized from the boxplot. It has the minimum and maximum points defined as Q1–1.5*IQR and Q3+1.5*IQR respectively. Any point outside this range is outlier.

![Image](https://miro.medium.com/max/481/1*8Eg6rRwfCNgXbY7ZX4C1ZA.png)

In [ ]:
boxplot = aq_df_2015.boxplot(column=['O3','PM10', 'PM2.5'], grid=False, rot=45, fontsize=15)

In [ ]:
boxplot = aq_df_2015.boxplot(column=['SO2', 'NO2'], grid=False, rot=45, fontsize=15)

In [ ]:
boxplot = aq_df_2015.boxplot(column=['O3'], grid=False, rot=45, fontsize=15)

So, we can say that outliers are present in our dataset.

#### Skewness Method to detect outliers

Several machine learning algorithms make the assumption that the data follow a normal (or Gaussian) distribution. This is easy to check with the skewness value, which explains the extent to which the data is normally distributed. Ideally, the skewness value should be between -1 and +1, and any major deviation from this range indicates the presence of extreme values.

The first line of code below prints the skewness value for the 'Pollution' variables, while the second line prints the summary statistics.

In [ ]:
# displaying skewness in features for the year 2015 
print(aq_df_2015.skew())
aq_df_2015.describe()

We can see that some features like $PM_{2.5}$, $PM_{10}$, $SO_2$, $NO_2$, $CO$, $O_3$, $RAIN$ and $WSPM$, having the value of *skewness* not in the optimal range, *indicating outliers*.

There are techniques to treat those outliers, one such technique is called IQR treatment of outliers will discuss in next section.

#### IQR Treatment of Outliers

Each dataset can be divided into quartiles. The first quartile point indicates that 25% of the data points are below that value whereas second quartile is considered as median point of the dataset. The interquartile method finds the outliers on numerical datasets by following the procedure below:

1. Find the first quartile, Q1.
2. Find the third quartile, Q3.
3. Calculate the IQR. IQR= Q3-Q1.
3. Define the normal data range with lower limit as Q1–1.5*IQR and upper limit as Q3+1.5*IQR.



In [ ]:
# Finding outliers
Q1 = aq_df_2015.quantile(0.25)
Q3 = aq_df_2015.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

The above output prints the IQR scores, which can be used to detect outliers. The code below generates an output with the 'True' and 'False' values. Points where the values are 'True' represent the presence of the outlier.

In [ ]:
# removing outliers using IQR values and redefining the dataset
df_out = aq_df_2015[~((aq_df_2015 < (Q1 - 1.5 * IQR)) |(aq_df_2015 > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

Here, we can see that the number of data points gets reduce from 8760 to 6031, meaning that we treat our data for outliers using IQR method. 

Now, we will check the **skew values** and see if the skewness of our features decreases or not after outlier treatment.

In [ ]:
print(df_out.skew())

Thus, we can say that we are able to reduce the skewness in our data as what we are getting when the data is having outliers in it.

### Autocorrelation Plot

Finally, we will talk about autocorrelation plot.

Autocorrelation refers to the degree of correlation between the values of the same variables across different observations in the data.  The concept of autocorrelation is most often discussed in the context of [time series](https://www.statisticssolutions.com/resources/directory-of-statistical-analyses/time-series-analysis) data in which observations occur at different points in time (e.g., temperature measured on different days of the month, hours of the day).  For example, one might expect the air temperature on the $1^{st}$ day of the month to be more similar to the temperature on the $2^{nd}$ day compared to the $31^{st}$ day.  If the temperature values that occurred closer together in time are, in fact, more similar than the temperature values that occurred farther apart in time, the data would be autocorrelated.

Autocorrelation can cause problems in conventional analyses (such as ordinary least squares regression) that assume independence of observations.

In a regression analysis, autocorrelation of the regression residuals can also occur if the model is incorrectly specified.  For example, if you are attempting to model a simple linear relationship but the observed relationship is non-linear (i.e., it follows a curved or U-shaped function), then the residuals will be autocorrelated.

In [ ]:
aq_df_na=aq_df.copy()
aq_df_na=aq_df_na.dropna()

In [ ]:
pd.plotting.autocorrelation_plot(aq_df_na['2014':'2016']['TEMP'])

Since our data is taken on a per hour basis, that's why the x-axis shows the number lagged hours.

From the above plot, we can infer that after every year the data is repeating the trend. This is also true as the particular season will come always at a definite time of the year.

**Note:** 8640 hours means one year. From this, we are able to infer the above visualization.

But here, in the above plot, there are so many lags, which is very difficult to visualize. So, it is a discussion point that can we condense our data into monthly basis so that we can easily and conveniently visualize it. 

That's what we are going to do in next section using `resample` function in `pandas` library.

In [ ]:
# sampling the data on monthly basis by taking mean for every month
aq_df_na['TEMP'].resample('1m').mean()

In [ ]:
pd.plotting.autocorrelation_plot(aq_df_na['2014':'2016']['TEMP'].resample('1m').mean())

Now, we can easily visualize here. At every 12 months (i.e. after every year) there is a repetition of peak meaning that the trend in temperature is repeating. 

Here, the two line shows the confidence interval (CI). The solid line indicates 90% CI and dash line represents 95% CI. The confidence interval shows the strength of autocorrelation in the data. The variation towards these CI lines represents more strength of autocorrelation.

In [ ]:
pd.plotting.autocorrelation_plot(aq_df_na['2014':'2016']['PM2.5'].resample('1m').mean())

Here, the data is within these lines, but that does not mean that there is no autocorrelation here. Maybe the strength of autocorrelation is less compared to that of temperature.

Moreover, we can observe trends after every year (i.e. after every 12 months).

With all these autocorrelation we are trying to infer that how our time series data will be better represented, what is the structure of it and what is pattern of your data like the trend, seasonality, etc.